# Synthetic Indoor Scene Dataset

5000 scenes with multiple objects from ShapeNet (chair, sofa, lamp, cabinet, table). From [ConvONet](https://github.com/autonomousvision/convolutional_occupancy_networks).

Size: 144GB  
Structure: 1000 rooms per dir  
Splits: [.75, .05, .2]  
Train: 0-750, Val: 751-800, Test: 801-999
```
- rooms_04
    - 00000000
    - ...
    - 00000999
    - test.lst
    - train.lst
    - val.lst
- ...
- rooms_08
```
Sample: `rooms_04/00000000`
```
- item_dict.npz    
- pointcloud0.ply  
- pointcloud/
- points_iou/
```

<font size='4'>`rooms_%02d`: the number indicates the **number of objects in the room**! </font>

#### Classes used from [ShapeNet](https://shapenet.org/taxonomy-viewer)
Classes  
`04256520`: sofa  
`03636649`: lamp  
`03001627`: chair  
`04379243`: table  
`02933112`: cabinet  

In [30]:
classes = ['04256520', '03636649', '03001627', '04379243', '02933112']
classes.sort()

In [31]:
classes

['02933112', '03001627', '03636649', '04256520', '04379243']

In [33]:
cls_map = {
    '04256520': 'sofa',
    '03636649': 'lamp',
    '03001627': 'chair',
    '04379243': 'table',
    '02933112': 'cabinet'
}

In [37]:
idx = lambda key: classes.index(key)
cls_idx = dict(sorted((idx(k), v) for k, v in cls_map.items()))
cls_idx[-1] = 'ground_plane'
cls_idx

{0: 'cabinet',
 1: 'chair',
 2: 'lamp',
 3: 'sofa',
 4: 'table',
 -1: 'ground_plane'}

In [1]:
import numpy as np
from plyfile import PlyElement, PlyData
import k3d
import trimesh

## `item_dict.npz`

In [12]:
item_dict = np.load('data/synthetic_room_dataset/rooms_04/00000000/item_dict.npz', allow_pickle=True)

In [21]:
for item in sorted(item_dict.files):
    print(item, item_dict[item].shape, '\n', item_dict[item])

bboxes (4, 2, 2) 
 [[[array([-0.21904938]) -0.22553027975898332]
  [array([0.16622645]) 0.09387186944046111]]

 [[array([-0.02461729]) 0.37421428136134205]
  [array([0.06721478]) 0.4653640989705788]]

 [[array([0.07332642]) 0.12255937464453248]
  [array([0.29560124]) 0.42709711771908837]]

 [[array([-0.22283809]) -0.37251215691330536]
  [array([-0.11733723]) -0.2672808741542804]]]
classes (4,) 
 ['03001627' '03636649' '02933112' '03636649']
n_objects () 
 4
objects (4,) 
 ['03001627/631671b636ab97d4da41a38d2e317241'
 '03636649/330b05e262c93e216078c74a96498820'
 '02933112/6c45346990283b7fa2e244117f013369'
 '03636649/b45b8df0557da6acc14d5c159bab8297']
room_idx () 
 0
scales (4,) 
 [0.33849483 0.25325549 0.39494988 0.39099373]
split () 
 train
wall_height (1,) 
 [0.25436147]
wall_height_range (2,) 
 [0.2 0.4]
wall_probabilities (4,) 
 [0.5 0.5 0.5 0.5]
wall_thickness () 
 0.01
walls (4,) 
 [1 1 0 1]
xz_groundplane_range (2,) 
 [array([0.72667497]) 1.0]
y_angle_range (2,) 
 [  0. 360.]
y_a

## `pointcloud0.ply`

In [8]:
pts_path = '/shared_data/data/nfs/Nina/cv/synthetic_room_dataset/rooms_04/00000000/pointcloud0.ply'

In [9]:
def load_pointcloud(in_file):
    plydata = PlyData.read(in_file)
    print(plydata)
    vertices = np.stack([
        plydata['vertex']['x'],
        plydata['vertex']['y'],
        plydata['vertex']['z']
    ], axis=1)
    return vertices

In [10]:
pc = load_pointcloud(pts_path)

ply
format binary_little_endian 1.0
comment github.com/mikedh/trimesh
element vertex 80978
property float x
property float y
property float z
element face 0
property list uchar int vertex_indices
end_header


In [104]:
# import trimesh

# mesh = trimesh.load(pts_path)
# scene = trimesh.Scene([mesh])
# scene.show()

# this is blank, as no faces

## `pointcloud` dir

10 randomly sampled point-clouds from a scene  
- pointcloud_00.npz
- ...
- pointcloud_09.npz

In [88]:
pointcloud_00 = np.load('data/synthetic_room_dataset/rooms_04/00000000/pointcloud/pointcloud_00.npz', allow_pickle=True)

In [91]:
for item in sorted(pointcloud_00.files):
    print(item, pointcloud_00[item].shape, '\n', pointcloud_00[item])

normals (80978, 3) 
 [[-0.1388  -0.985   -0.10297]
 [-0.1604   0.95    -0.2666 ]
 [ 0.10736  0.02011 -0.994  ]
 ...
 [-0.       1.       0.     ]
 [-0.       1.       0.     ]
 [-0.       1.       0.     ]]
points (80978, 3) 
 [[-0.10645 -0.4841   0.03632]
 [ 0.0974  -0.2668  -0.1624 ]
 [ 0.141   -0.3667  -0.1544 ]
 ...
 [-0.3486  -0.49    -0.0475 ]
 [ 0.3303  -0.49     0.2766 ]
 [ 0.1202  -0.49     0.01799]]
semantics (80978,) 
 [ 1  1  1 ... -1 -1 -1]


-1 is the ground plane

In [93]:
np.unique(pointcloud_00['semantics'])

array([-1,  0,  1,  2])

## `points_iou` dir

In [96]:
points_iou = np.load('data/synthetic_room_dataset/rooms_04/00000000/points_iou/points_iou_00.npz', allow_pickle=True)

In [97]:
for item in sorted(points_iou.files):
    print(item, points_iou[item].shape, '\n', points_iou[item])

occupancies (12500,) 
 [ 0 32  0 ...  0  0  0]
points (100000, 3) 
 [[-0.05865  0.1582  -0.14   ]
 [-0.247   -0.3103  -0.3682 ]
 [-0.5054   0.266   -0.4922 ]
 ...
 [ 0.373    0.4866  -0.2375 ]
 [-0.3718   0.4011   0.4883 ]
 [ 0.2644   0.429    0.1742 ]]
semantics (100000,) 
 [7 7 7 ... 7 7 7]
z_scale () 
 0


In [98]:
np.unique(points_iou['semantics'])

array([-1,  0,  1,  2,  7])

# ShapeNet

[ShapeNet](https://shapenet.org/) - richly-annotated, large-scale dataset of 3D shapes

ShapeNetCore is a subset of the full ShapeNet dataset with single clean 3D models and manually verified category and alignment annotations. It covers 55 common object categories with about 51,300 unique 3D models. The 12 object categories of PASCAL 3D+, a popular computer vision 3D benchmark dataset, are all covered by ShapeNetCore.

```
{
    "synsetId": "02690373",
    "name": "airliner",
    "children": [
      "03809312",
      "04583620"
    ],
    "numInstances": 1490
},
```

Structure
```
- synsetId
    - hash_id
        - images
            - texture0.jpg
        - models
            - model_normalized.json
            - model_normalized.obj 
            - model_normalized.surface.binvox
            - model_normalized.mtl
            - model_normalized.solid.binvox
```

With Pytorch3D - https://pytorch3d.readthedocs.io/en/latest/_modules/pytorch3d/datasets/shapenet/shapenet_core.html